In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, LinearRegression
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor

RSEED = 42

In [ ]:
df_train_prepro = pd.read_csv('data/preprocessed_train_data_with_date_hol_concat.csv')

In [ ]:
y = df_train_prepro['target']
X = df_train_prepro.drop(columns=['target'])

In [ ]:
# Define the categorical features
num_col = ['duration','dep_temp', 'dep_precip', 'dep_wind', 'arr_temp',
       'arr_precip', 'arr_wind', 'holiday_length']
cat_col = ['departure_point', 'arrival_point', 'flight_status', 'aircraft_code','dep_hour',
       'dep_day', 'dep_month', 'dep_dayofweek', 'dep_quarter', 'dep_season',
       'dep_is_weekend', 'dep_time_of_day', 'arr_hour', 'arr_day', 'arr_month',
       'arr_dayofweek', 'arr_quarter', 'arr_season', 'arr_is_weekend',
       'arr_time_of_day', 'route', 'is_holiday', 'Country', 'City']

In [ ]:
# Use sparse output for OneHotEncoder to save memory
encoder = OneHotEncoder(handle_unknown='ignore')
scaler = StandardScaler()

# Fit and transform categorical columns (sparse matrix)
X_cat_sparse = encoder.fit_transform(X[cat_col])

# Scale only the numerical columns and convert to float32
X_num_scaled = scaler.fit_transform(X[num_col]).astype(np.float32)

# Convert sparse matrix to float32 and combine with numerical features
from scipy import sparse
X_encoded_scaled = sparse.hstack([X_num_scaled, X_cat_sparse.astype(np.float32)]).tocsr()

# Split the encoded and scaled data
X_train_1, X_train_2, y_train_1, y_train_2 = train_test_split(
    X_encoded_scaled, y, stratify=y, test_size=0.2, random_state=RSEED
)

In [ ]:
# Get indices for the split
train_idx, test_idx = train_test_split(
    np.arange(len(X)), stratify=y, test_size=0.2, random_state=RSEED
)

X_train_1_raw = X.iloc[train_idx]
X_train_2_raw = X.iloc[test_idx]
y_train_1_raw = y.iloc[train_idx]
y_train_2_raw = y.iloc[test_idx]

# Get categorical column indices for CatBoost
cat_features_idx = [X.columns.get_loc(col) for col in cat_col]

XGB RandomisedSearchCV

Find the best parameters for XGBRegressor

In [ ]:
# Fit model to training data
xgb_best = XGBRegressor(objective='reg:squarederror', random_state=42)

In [ ]:
#hyperparameter grid
xgb_param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 4, 5, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8],
    'colsample_bytree': [0.6, 0.8],
    'gamma': [0, 0.1, 0.2, 0.5],
    'min_child_weight': [1, 3, 5],
    'scale_pos_weight': [1, 2]
}

In [ ]:
xgb_search = GridSearchCV(XGBRegressor(objective='reg:squarederror', random_state=42), xgb_param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
xgb_search.fit(X_train_1, y_train_1)
best_xgb = xgb_search.best_estimator_

In [ ]:
#hyperparameter grid
xgb_param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 4, 5, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.5, 1],
    'min_child_weight': [1, 3, 5, 7],
    'scale_pos_weight': [1, 2, 3]
}

xgb_ransearcv_best = RandomizedSearchCV(estimator=xgb_best, param_distributions=xgb_param_grid, 
                          n_iter=100, scoring='accuracy', cv=3, 
                          verbose=1, random_state=42, n_jobs=-1)
xgb_ransearcv_best.fit(X_train_1, y_train_1)

In [ ]:
rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5]
}

xgb_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

ridge_param_grid = {
    'alpha': [0.1, 1.0, 10.0]
}

lgbm_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

In [ ]:
from sklearn.model_selection import GridSearchCV

# Random Forest
rf_search = GridSearchCV(RandomForestRegressor(random_state=42), rf_param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
rf_search.fit(X_train_1, y_train_1)
best_rf = rf_search.best_estimator_

# XGBoost
xgb_search = GridSearchCV(XGBRegressor(objective='reg:squarederror', random_state=42), xgb_param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
xgb_search.fit(X_train_1, y_train_1)
best_xgb = xgb_search.best_estimator_

# Ridge
ridge_search = GridSearchCV(Ridge(random_state=42), ridge_param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
ridge_search.fit(X_train_1, y_train_1)
best_ridge = ridge_search.best_estimator_

# LightGBM
lgbm_search = GridSearchCV(LGBMRegressor(random_state=42), lgbm_param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
lgbm_search.fit(X_train_1, y_train_1)
best_lgbm = lgbm_search.best_estimator_

Create Base models + Meta model

In [ ]:
# Base models
random_forest = RandomForestRegressor(random_state=42)
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)
ridge = Ridge(random_state=42)
knn = KNeighborsRegressor()
#lgbm = LGBMRegressor(random_state=42)
catboost = CatBoostRegressor(verbose=0, random_state=42)

In [ ]:
# Meta-model to combine all the base models
linear_model = LinearRegression()

In [ ]:
# Fit all the base estimators on the 1st half of the train dataset

#Fit Random Forest
rf_model = random_forest.fit(X_train_1, y_train_1)

In [ ]:
#Fit XGBoost
xgb_model = xgb.fit(X_train_1, y_train_1)

In [ ]:
# Fit Ridge Convert sparse matrix to dense for Ridge regression
ridge_model = Ridge(random_state=42).fit(X_train_1.toarray(), y_train_1)

In [ ]:
# fit KNN KNeighborsRegressor does not support sparse input, so convert to dense
knn_model = knn.fit(X_train_1, y_train_1)

In [ ]:
#lgbm_model = lgbm.fit(X_train_1, y_train_1)

In [ ]:
# Drop datetime columns that CatBoost cannot handle
datetime_cols = ['departure_time', 'arrival_time', 'departure_date', 'arrival_date']
X_train_1_raw_catboost = X_train_1_raw.drop(columns=datetime_cols)

# Update categorical feature indices for the new dataframe
cat_features_idx_catboost = [X_train_1_raw_catboost.columns.get_loc(col) for col in cat_col if col in X_train_1_raw_catboost.columns]

catboost.fit(X_train_1_raw_catboost, y_train_1_raw, cat_features=cat_features_idx_catboost)

In [ ]:
# Predict the values from the base estimators with the second half of the train dataset
rf_pred = rf_model.predict(X_train_2)
xgb_pred = xgb_model.predict(X_train_2)
ridge_pred = ridge_model.predict(X_train_2)
knn_pred = knn_model.predict(X_train_2)
#lgbm_pred = lgbm_model.predict(X_train_2)
catboost_pred = catboost.predict(X_train_2_raw.drop(columns=datetime_cols))

#lgbm_pred = lgbm_model.predict(X_train_2)

# Combine base model predictions for meta-model input
combine_X_pred_test = pd.concat([
	pd.DataFrame(rf_pred),
	pd.DataFrame(xgb_pred),
	pd.DataFrame(ridge_pred),
    pd.DataFrame(knn_pred),
    #pd.DataFrame(lgbm_pred),
    pd.DataFrame(catboost_pred)
], axis=1)

In [ ]:
# Fit the final estimator on the combined probabilities and target values
linear_model.fit(combine_X_pred_test, y_train_2)

In [ ]:
# Predict with meta-model
y_pred = linear_model.predict(combine_X_pred_test)

In [ ]:
# Ensure predictions are non-negative
y_pred[y_pred < 0] = 0
y_train_2 = y_train_2.clip(lower=0)

In [ ]:
# Evaluate the model
mse = mean_squared_error(y_train_2, y_pred)
r2 = r2_score(y_train_2, y_pred)
rmse = np.sqrt(mean_squared_error(y_train_2, y_pred))
print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2}')
print(f"Stacking RMSE: {rmse:.2f}")

In [ ]:
# Get feature importances from each base model and display them

# Random Forest feature importances
rf_importances = rf_model.feature_importances_

# XGBoost feature importances
xgb_importances = xgb_model.feature_importances_

# KNN does not provide feature importances, so skip it

# Ridge feature coefficients (absolute value for importance)
ridge_importances = np.abs(ridge_model.coef_)

# LightGBM feature importances
#lgbm_importances = lgbm_model.feature_importances_

# Get encoded categorical column names
encoded_cat_cols = encoder.get_feature_names_out(cat_col)

# Feature names
feature_names = num_col + list(encoded_cat_cols)

# Create a DataFrame for each model's importances
importances_df = pd.DataFrame({
    'feature': feature_names,
    'RandomForest': rf_importances,
    'XGBoost': xgb_importances,
    'Ridge': ridge_importances,
    #'LightGBM': lgbm_importances
})

# Show top 15 features by average importance across models
importances_df['avg_importance'] = importances_df[['RandomForest', 'XGBoost', 'Ridge']].mean(axis=1)
importances_df.sort_values('avg_importance', ascending=False)